In [1]:
import os
import glob
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad

# Prepare input for RCTD

In [ ]:
def preparePuckForRCTD(date, puck, path, adata=None):
    if not adata:
        try:
            adata = read_adata(date, puck)
        except:
            adata = read_old_adata(date, puck)
        
    BeadLocationsForR = adata.obs.copy().reset_index()
    BeadLocationsForR.columns = ['barcodes', 'xcoord', 'ycoord']
    
    barcodes = BeadLocationsForR[['barcodes']].to_numpy().squeeze()
    if barcodes[0][-1]=='1':
        barcodes = [x[:-2] for x in barcodes]
    BeadLocationsForR['barcodes'] = barcodes
    
    BeadLocationsForR.to_csv(f"{path}/{date}_{puck}_BeadLocationsForR.csv", index=False)
    
    genes = adata.var.copy().reset_index()[['gene']].to_numpy().squeeze()
    MappedDGEForR = pd.DataFrame.sparse.from_spmatrix(adata.T.X, genes, barcodes)
    
    MappedDGEForR.to_csv(f"{path}/{date}_{puck}_MappedDGEForR.csv", chunksize=1000)
    return BeadLocationsForR, MappedDGEForR

In [ ]:
samples = [
    {'run': '250828_RL_PCGC',
    'puck': 'C85_7',
    'date': '2025-07-25',
    'puck_name': 'Puck_250122_07'},
    
    {'run': '250828_RL_PCGC',
    'puck': 'C85_14',
    'date': '2025-07-25',
    'puck_name': 'Puck_250122_14'},
    
    {'run': '250828_RL_PCGC',
    'puck': 'C53_23',
    'date': '2025-07-25',
    'puck_name': 'Puck_250707_23'},
    
    ]

In [ ]:
# process in situ sequencing file containing bead locations and curio outputs for RCTD
for sample in samples:
    run  = sample['run']
    puck = sample['puck']
    date = sample['date']
    puck_name = sample['puck_name']

    print(f"Processing {puck} from {run}...")

    adata_path = f"/data/liulab/software/curioseeker-v3.0.0/pipeline_outputs/{run}/results/OUTPUT/{puck}/{puck}_anndata.h5ad"
    bcxy_path  = f"/data/liulab/software/curioseeker-v3.0.0/pipeline_outputs/{run}/results/OUTPUT/{puck}/{puck}_MatchedBeadLocation.csv"

    adata = sc.read_h5ad(adata_path)
    bcxy  = pd.read_csv(bcxy_path)
    bcxy.columns = ["bc","x","y"]

    adata.obs = adata.obs.merge(bcxy.set_index("bc"), left_index=True, right_index=True)
    adata.obs = adata.obs[["x","y"]]

    if "gene" not in adata.var.columns or adata.var.shape[1] != 1:
        adata.var = pd.DataFrame({"gene": adata.var_names.astype(str)})

    BeadLocationsForR, MappedDGEForR = preparePuckForRCTD(
        f"curio_{date}", puck_name, path="/data/liulab/ryan/GCPC/RCTD_input", adata=adata
    )

# Merge RCTD outputs w/ Curio H5ad

In [2]:
samples = [
    {
        "run": "rl_batista_filemove",
        "puck": "C85_32",
        "date": "2025-04-30",
        "puck_name": "Puck_250122_32",
    },
    {
        "run": "rl_batista_filemove",
        "puck": "C85_33",
        "date": "2025-04-30",
        "puck_name": "Puck_250122_33",
    },
    {
        'run': '250828_RL_PCGC',
        'puck': 'C85_7',
        'date': '2025-07-25',
        'puck_name': 'Puck_250122_07'
    },
    
    {
        'run': '250828_RL_PCGC',
        'puck': 'C85_14',
        'date': '2025-07-25',
        'puck_name': 'Puck_250122_14'
    }
]

In [3]:
def process_RCTD(adata_file, rctd_file, weights_file):
    '''
    Takes RCTD cell type calling outputs and appends cell type data to AnnData object.
    
    Parameters:
    - adata_file: AnnData object
    - rctd_file: tsv file with cell type information that maps barcode to cell type
    - weights_file: tsv file that maps barcodes to cell type weight
    
    Returns:
    - `"spot_class"` : RCTD classification of spot.
    - `"first_type"` : top predicted cell type per spot.
    - `"second_type"` : second predicted cell type per spot.
    - `"first_weight"` : weight (proportion) for the top predicted cell type.
    - `"second_weight"` : weight (proportion) for the second predicted cell type.
    '''
    rctd_file.set_index("Unnamed: 0", inplace=True)
    adata_file.obs = adata_file.obs.merge(
        rctd_file[["spot_class", "first_type", "second_type"]],
        left_index=True,
        right_index=True,
        how="left",
    )

    first_type = adata_file.obs.first_type.tolist()
    second_type = adata_file.obs.second_type.tolist()
    bcs = adata_file.obs.index.tolist()

    weights_file.set_index("Unnamed: 0", inplace=True)

    celltype = list(weights_file.columns)
    weights_array = weights_file.to_numpy()
    bc2celltype2weights = {}
    for i, bc in enumerate(weights_file.index.tolist()):
        bc2celltype2weights[bc] = {
            celltype[j]: weights_array[i][j] for j in range(len(celltype))
        }

    first_weight = []
    second_weight = []
    for i, bc in enumerate(bcs):
        if first_type[i] in celltype:
            first_weight.append(bc2celltype2weights[bc][first_type[i]])
            second_weight.append(bc2celltype2weights[bc][second_type[i]])
        else:
            first_weight.append(first_type[i])
            second_weight.append(second_type[i])

    adata_file.obs["first_weight"] = first_weight
    adata_file.obs["second_weight"] = second_weight
    return adata_file

In [4]:
for sample in samples:
    run_name    = sample["run"]
    puck_id     = sample["puck"]
    sample_date = sample["date"]
    puck_details= sample["puck_name"]

    adata = sc.read_h5ad(
        f"/data/liulab/software/curioseeker-v3.0.0/pipeline_outputs/"
        f"{run_name}/results/OUTPUT/{puck_id}/{puck_id}_anndata.h5ad"
    )
    bcxy = pd.read_csv(
        f"/data/liulab/software/curioseeker-v3.0.0/pipeline_outputs/"
        f"{run_name}/results/OUTPUT/{puck_id}/{puck_id}_MatchedBeadLocation.csv"
    )
    bcxy.columns = ["bc", "x", "y"]
    adata.obs = adata.obs.merge(bcxy.set_index("bc"), left_index=True, right_index=True)
    adata.var.columns = ["gene"]

    base_dir     = "/data/liulab/kevin/RCTD_output"
    dir_prefix   = f"curio_{sample_date}_{puck_details}_RCTD"
    file_prefix  = f"curio_{sample_date}_{puck_details}"

    candidate_dirs = [os.path.join(base_dir, f"{dir_prefix}_RCTD"),
                      os.path.join(base_dir, dir_prefix),
                      os.path.join(base_dir, file_prefix)]

    rctd_file = weights_file = None
    chosen_dir = None

    for rdir in candidate_dirs:
        if not os.path.isdir(rdir):
            continue
        rf = os.path.join(rdir, f"{file_prefix}_output.csv")
        wf = os.path.join(rdir, f"{file_prefix}_norm_weights.csv")
        if os.path.exists(rf) and os.path.exists(wf):
            rctd_file, weights_file, chosen_dir = rf, wf, rdir
            break
        # Fallback: glob for the two files if exact names don’t match
        cand_rf = glob.glob(os.path.join(rdir, "*_output.csv"))
        cand_wf = glob.glob(os.path.join(rdir, "*_norm_weights.csv"))
        if cand_rf and cand_wf:
            rctd_file, weights_file, chosen_dir = cand_rf[0], cand_wf[0], rdir
            break

    if rctd_file is None:
        tried = "\n  ".join(candidate_dirs)
        raise FileNotFoundError(
            f"RCTD outputs not found for {puck_id} ({sample_date}). Tried:\n  {tried}"
        )

    rctd    = pd.read_csv(rctd_file)
    weights = pd.read_csv(weights_file)

    processed_results = process_RCTD(adata, rctd, weights)
    var_name = f"{puck_id}_adata"
    globals()[var_name] = processed_results

    out_filename = f"{puck_id}_processed.h5ad"
    processed_results.write(out_filename)
    
    print(f"Loaded RCTD as `{var_name}` from file: {rctd_file}")

Loaded RCTD as `C85_32_adata` from file: /data/liulab/kevin/RCTD_output/curio_2025-04-30_Puck_250122_32_RCTD_RCTD/curio_2025-04-30_Puck_250122_32_RCTD_output.csv
Loaded RCTD as `C85_33_adata` from file: /data/liulab/kevin/RCTD_output/curio_2025-04-30_Puck_250122_33_RCTD_RCTD/curio_2025-04-30_Puck_250122_33_RCTD_output.csv
Loaded RCTD as `C85_7_adata` from file: /data/liulab/kevin/RCTD_output/curio_2025-07-25_Puck_250122_07_RCTD_RCTD/curio_2025-07-25_Puck_250122_07_RCTD_output.csv
Loaded RCTD as `C85_14_adata` from file: /data/liulab/kevin/RCTD_output/curio_2025-07-25_Puck_250122_14_RCTD/curio_2025-07-25_Puck_250122_14_output.csv
